In [10]:
# External libraries
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from scipy import stats
from scipy.optimize import curve_fit

In [8]:
def Round(value, sf):
    
    if value == 0.00:
        return "0"
    elif math.isnan(value):
        return "NaN"
    else:
        # Determine the order of magnitude
        magnitude = math.floor(math.log10(abs(value))) + 1
        # Calculate the scale factor
        scale_factor = sf - magnitude
        # Truncate the float to the desired number of significant figures
        truncated_value = math.trunc(value * 10 ** scale_factor) / 10 ** scale_factor
        # Convert the truncated value to a string
        truncated_str = str(truncated_value).rstrip('0').rstrip('.')

        return truncated_str

In [45]:
# Stats for histograms tends to assume a normal distribution
# ROOT does the same thing with TH1
def GetBasicStats(data, xmin, xmax):

    filtered_data = data[(data >= xmin) & (data <= xmax)]  # Filter data within range

    N = np.sum(filtered_data)                      
    mean = np.mean(filtered_data)  
    meanErr = stats.sem(filtered_data) # Mean error (standard error of the mean from scipy)
    stdDev = np.std(filtered_data) # Standard deviation
    stdDevErr = np.sqrt(stdDev**2 / (2*N)) # Standard deviation error assuming normal distribution
    underflows = len(data[data < xmin]) # Number of underflows
    overflows = len(data[data > xmax])

    return N, mean, meanErr, stdDev, stdDevErr, underflows, overflows

In [6]:
def GetBinCentres(binEdges): 
    return (binEdges[:-1] + binEdges[1:]) / 2

In [4]:
# The Gaussian function
def GausFunc(x, norm, mu, sigma):
    return norm * np.exp(-((x - mu) / (2 * sigma)) ** 2)

In [5]:
# def GausPar(counts, binEdges, fitMin, fitMax, norm=1.0, mu=0.0, sigma=1.0):
#     # Calculate bin centers
#     binCentres = GetBinCentres(binEdges) 
#     # Fit the Gaussian function to the histogram data
#     params, covariance = curve_fit(GausFunc, binCentres[(binCentres >= fitMin) & (binCentres <= fitMax)], counts, p0=[norm, mu, sigma])
#     # Extract parameters from the fitting
#     norm, mu, sigma = params
#     return norm, mu, sigma

In [1]:
def ScientificNotation(ax):

    # Scientific notation
    if (ax.get_xlim()[1] > 9.999e3) or (ax.get_xlim()[1] < 9.999e-3):
        ax.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
        ax.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
        ax.xaxis.offsetText.set_fontsize(14)
    if (ax.get_ylim()[1] > 9.999e3) or (ax.get_ylim()[1] < 9.999e-3):
        ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
        ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
        ax.yaxis.offsetText.set_fontsize(14)

    return

In [ ]:
def Hist1D(x, nbins=100, xmin=-1.0, xmax=1.0):

    # Create 2D histogram
    counts, x_edges = np.histogram(x, bins=nbins, range=[xmin, xmax])

    return counts, x_edges

In [11]:
def Plot1D(data, nbins=100, xmin=-1.0, xmax=1.0, title=None, xlabel=None, ylabel=None, fout="Images/hist.png", legPos="best", stats=True, underOver=False, errors=False, NDPI=300):
    
    data = np.array(data)
    # data = np.flatten(data)

    # Create figure and axes
    fig, ax = plt.subplots()

    # Plot the histogram with outline
    counts, bin_edges, _ = ax.hist(data, bins=nbins, range=(xmin, xmax), histtype='step', edgecolor='black', linewidth=1.0, fill=False, density=False)

    # Set x-axis limits
    ax.set_xlim(xmin, xmax)

    # # # Calculate statistics
    N, mean, meanErr, stdDev, stdDevErr, underflows, overflows = GetBasicStats(data, xmin, xmax)
    # # N, mean, meanErr, stdDev, stdDevErr = str(N), Round(mean, 3), Round(meanErr, 1), Round(stdDev, 3), Round(stdDevErr, 1) 

    # # Create legend text
    legend_text = f"Entries: {N}\nMean: {Round(mean, 3)}\nStd Dev: {Round(stdDev, 3)}"
    # if errors: legend_text = f"Entries: {N}\nMean: {Round(mean, 3)}$\pm${Round(meanErr, 1)}\nStd Dev: {Round(stdDev, 3)}$\pm${Round(stdDevErr, 1)}"
    # if underOver: legend_text += f"\nUnderflows: {underflows}\nOverflows: {overflows}"

    # # legend_text = f"Entries: {N}\nMean: {Round(mean, 3)}$\pm${Round(meanErr, 1)}\nStd Dev: {Round(stdDev, 3)}$\pm${Round(stdDev, 1)}"

    # Add legend to the plot
    if stats: ax.legend([legend_text], loc=legPos, frameon=False, fontsize=13)

    ax.set_title(title, fontsize=15, pad=10)
    ax.set_xlabel(xlabel, fontsize=13, labelpad=10) 
    ax.set_ylabel(ylabel, fontsize=13, labelpad=10) 

    # Set font size of tick labels on x and y axes
    ax.tick_params(axis='x', labelsize=13)  # Set x-axis tick label font size
    ax.tick_params(axis='y', labelsize=13)  # Set y-axis tick label font size

    ScientificNotation(ax) 
    
    # if (ax.get_xlim()[1] > 9.999e3) or (ax.get_xlim()[1] < 9.999e-3):
    #     ax.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    #     ax.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    #     ax.xaxis.offsetText.set_fontsize(13)
    # if (ax.get_ylim()[1] > 9.999e3) or (ax.get_ylim()[1] < 9.999e-3):
    #     ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    #     ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    #     ax.yaxis.offsetText.set_fontsize(13)

    # Save the figure
    plt.show() 

    plt.savefig(fout, dpi=NDPI, bbox_inches="tight")
    print("\n---> Written:\n\t", fout)

    # Clean up memory
    plt.close()

In [42]:
def Plot1DWithGausFit(data, nbins=100, xmin=-1.0, xmax=1.0, norm=1.0, mu=0.0, sigma=1.0, fitMin=-1.0, fitMax=1.0, title=None, xlabel=None, ylabel=None, fout="Images/hist.png", legPos="best", stats=True, peak=False, underOver=False, errors=False, NDPI=300):
    
    data = np.array(data)
    
    # Create figure and axes
    fig, ax = plt.subplots()

    # Plot the histogram with outline
    counts, binEdges, _ = ax.hist(data, bins=nbins, range=(xmin, xmax), histtype='step', edgecolor='black', linewidth=1.0, fill=False, density=False)

    # Set x-axis limits
    ax.set_xlim(xmin, xmax)
    
    ####################
    # Construct gaussian
    ####################
    
    # Calculate bin centers
    binCentres = GetBinCentres(binEdges) 
    
    # Filter bin_centres and counts based on fitMin and fitMax
    valid = (binCentres >= fitMin) & (binCentres <= fitMax)
    binCentresFit = binCentres[valid]
    countsFit = counts[valid]
    
    # Fit the Gaussian function to the histogram data
    params, covariance = curve_fit(GausFunc, binCentresFit, countsFit, p0=[norm, mu, sigma])
    # Extract parameters from the fitting
    norm, mu, sigma = params
    # Plot the Gaussian curve
    ax.plot(binCentresFit, GausFunc(binCentresFit, norm, mu, sigma), color="red")
    
    # Calculate statistics
    N, mean, meanErr, stdDev, stdDevErr, underflows, overflows = GetBasicStats(data, xmin, xmax)

    # Calculate chi-squared
    y_observed = countsFit
    y_expected = GausFunc(binCentresFit, norm, mu, sigma)
    chi2 = np.sum( (y_observed - y_expected)**2 / y_expected)
    ndf = len(y_observed) - len(params)  # number of degrees of freedom
    chi2ndf = chi2 / ndf
    
    print("chi2 =", chi2)
    print("chi2/ndf =", chi2ndf)

    # Create legend text
    # legendText = [f"Entries: {N}\nMean: {Round(mean, 3)}\nStd Dev: {Round(stdDev, 3)}", f"\$\chi^{2}$/ndf: {Round(chi2ndf, 3)}\nNorm: {Round(norm,3)}\n$\mu$: {Round(mu,3)}\n$\sigma$: {Round(sigma, 3)}"]
    legendText = [f"Entries: {N}\nMean: {Round(mean, 3)}\nStd Dev: {Round(stdDev, 3)}", f"\nNorm: {Round(norm,3)}\n$\mu$: {Round(mu,3)}\n$\sigma$: {Round(sigma, 3)}"]

    
    # legend_text
    # if errors: legend_text = f"Entries: {N}\nMean: {Round(mean, 3)}$\pm${Round(meanErr, 1)}\nStd Dev: {Round(stdDev, 3)}$\pm${Round(stdDevErr, 1)}"
    # if errors: legend_text = f"Entries: {N}\nMean: {Round(mean, 4)}$\pm${Round(meanErr, 1)}\nStd Dev: {Round(stdDev, 4)}$\pm${Round(stdDevErr, 1)}"
    # # if peak: legend_text += f"\nPeak: {Round(peak, 4)}$\pm${Round(peakErr, 1)}"
    # if underOver: legend_text += f"\nUnderflows: {underflows}\nOverflows: {overflows}"

    # legend_text = f"Entries: {N}\nMean: {Round(mean, 3)}$\pm${Round(meanErr, 1)}\nStd Dev: {Round(stdDev, 3)}$\pm${Round(stdDev, 1)}"

    # Add legend to the plot
    if stats: ax.legend(legendText, loc=legPos, frameon=False, fontsize=13)

    ax.set_title(title, fontsize=15, pad=10)
    ax.set_xlabel(xlabel, fontsize=13, labelpad=10) 
    ax.set_ylabel(ylabel, fontsize=13, labelpad=10) 

    # Set font size of tick labels on x and y axes
    ax.tick_params(axis='x', labelsize=13)  # Set x-axis tick label font size
    ax.tick_params(axis='y', labelsize=13)  # Set y-axis tick label font size

    # Scientific notation
    ScientificNotation(ax) 

    # Save the figure
    plt.show() # (fout, dpi=NDPI, bbox_inches="tight")

    plt.savefig(fout, dpi=NDPI, bbox_inches="tight")
    print("\n---> Written:\n\t", fout)

    # Clean up memory
    plt.close()

In [36]:
def Hist2D(x, y, nbinsX=100, xmin=-1.0, xmax=1.0, nbinsY=100, ymin=-1.0, ymax=1.0):

    # Filter out empty entries from x and y
    valid_indices = [i for i in range(len(x)) if np.any(x[i]) and np.any(y[i])]

    # Extract valid data points based on the indices
    x = [x[i] for i in valid_indices]
    y = [y[i] for i in valid_indices]

    # Check if the input arrays are not empty and have the same length
    if len(x) == 0 or len(y) == 0:
        print("Input arrays are empty.")
        return
    if len(x) != len(y):
        print("Input arrays x and y have different lengths.")
        return

    # Create 2D histogram
    hist, x_edges, y_edges = np.histogram2d(x, y, bins=[nbinsX, nbinsY], range=[[xmin, xmax], [ymin, ymax]])

    return hist # , x_edges, y_edges

In [35]:
def Plot2D(x, y, nbinsX=100, xmin=-1.0, xmax=1.0, nbinsY=100, ymin=-1.0, ymax=1.0, title=None, xlabel=None, ylabel=None, fout="Images/hist.png", log=False, cb=True, NDPI=300):

    # Create 2D histogram
    hist, x_edges, y_edges = Hist2D(x, y, nbinsX, xmin, xmax, nbinsY, ymin, ymax) 

    # Set up the plot
    fig, ax = plt.subplots()

    norm = colors.Normalize(vmin=0, vmax=np.max(hist))  
    if log: norm = colors.LogNorm(vmin=1, vmax=np.max(hist)) 

    # Plot the 2D histogram
    im = ax.imshow(hist.T, cmap='inferno', extent=[xmin, xmax, ymin, ymax], aspect='auto', origin='lower', norm=norm)  # , vmax=np.max(hist), norm=colors.LogNorm())
    # im = ax.imshow(hist.T, extent=[xmin, xmax, ymin, ymax], aspect='auto', origin='lower', vmax=np.max(hist))

    # Add colourbar
    if cb: plt.colorbar(im)

    plt.title(title, fontsize=16, pad=10)
    plt.xlabel(xlabel, fontsize=14, labelpad=10)
    plt.ylabel(ylabel, fontsize=14, labelpad=10)

    # Scientific notation
    ScientificNotation(ax) 
    
    # if (ax.get_xlim()[1] > 9.999e3) or (ax.get_xlim()[1] < 9.999e-3):
    #     ax.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    #     ax.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    #     ax.xaxis.offsetText.set_fontsize(14)
    # if (ax.get_ylim()[1] > 9.999e3) or (ax.get_ylim()[1] < 9.999e-3):
    #     ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    #     ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    #     ax.yaxis.offsetText.set_fontsize(14)

    # plt.savefig(fout, dpi=NDPI, bbox_inches="tight")
    # print("\n---> Written:\n\t", fout)
    plt.show()

    plt.savefig(fout, dpi=NDPI, bbox_inches="tight")
    print("\n---> Written:\n\t", fout)

    # Clean up memory
    plt.close()